# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]


In [2]:
text[4:50]

"Szyslak: (INTO PHONE) Moe's Tavern. Where the "

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (10, 20)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 10 to 20:
Moe_Szyslak: Ah, isn't that nice. Now, there is a politician who cares.
Barney_Gumble: If I ever vote, it'll be for him. (BELCH)


Barney_Gumble: Hey Homer, how's your neighbor's store doing?
Homer_Simpson: Lousy. He just sits there all day. He'd have a great job if he didn't own the place. (CHUCKLES)
Moe_Szyslak: (STRUGGLING WITH CORKSCREW) Crummy right-handed corkscrews! What does he sell?
Homer_Simpson: Uh, well actually, Moe...
HOMER_(CONT'D: I dunno.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
text[:2000]

'Moe_Szyslak: (INTO PHONE) Moe\'s Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I\'ll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I\'m gonna catch you, and I\'m gonna carve my name on your back with an ice pick.\nMoe_Szyslak: What\'s the matter Homer? You\'re not your normal effervescent self.\nHomer_Simpson: I got my problems, Moe. Give me another one.\nMoe_Szyslak: Homer, hey, you should not drink to forget your problems.\nBarney_Gumble: Yeah, you should only drink to enhance your social skills.\n\n\nMoe_Szyslak: Ah, isn\'t that nice. Now, there is a politician who cares.\nBarney_Gumble: If I ever vote, it\'ll be for him. (BELCH)\n\n\nBarney_Gumble: Hey Homer, how\'s your neighbor\'s store doing?\nHomer_Simpson: Lousy. He just sits there all day. He\'d have a great job if he didn\'

In [17]:
len(text)

305189

In [19]:
text[0]

'M'

In [3]:
from collections import Counter

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
     
    
    
    return (vocab_to_int, int_to_vocab)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)


Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [5]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punctuation_dic = {'.': '||period||',
     ',': '||comma||',
     '"': '||quotation_mark||',
     ';': '||semicolon||',
     '!': '||exclamation_mark||',
     '?': '||question_mark||',
     '(': '||left_parentheses||',
     ')': '||right_parentheses||',
     '--': '||dash||',
     '\n': '||new_line||'}
    return punctuation_dic

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)


Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [10]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    #print(input.shape)
    return (input, targets, learning_rate)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)


Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [11]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    #keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    #drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    cell = tf.contrib.rnn.MultiRNNCell([lstm])
    initial_state = tf.identity(cell.zero_state(batch_size, tf.float32),name = 'initial_state')
    return (cell, initial_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


In [60]:
cell_c, ista =  get_init_cell(256, 2)

In [65]:
print(cell_c.state_size)

(LSTMStateTuple(c=2, h=2),)


In [66]:
print(cell_c.output_size)

2


In [61]:
print(ista)

Tensor("initial_state:0", shape=(1, 2, 256, 2), dtype=float32)


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embed_size = embed_dim
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_size), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [13]:

def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    print(inputs.shape)
    
    #print(cell.state_size)
    Outputs, final_state = tf.nn.dynamic_rnn(cell,inputs,dtype = tf.float32)
    final_state = tf.identity(final_state,name = 'final_state')
    print(Outputs.get_shape())
    return (Outputs, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)


(?, ?, 256)
(?, ?, 256)
Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [69]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    embed = get_embed(input_data, vocab_size, embed_dim)
    Outputs, final_state = build_rnn(cell, embed)
    print(Outputs.shape)
    predictions = tf.contrib.layers.fully_connected(Outputs, vocab_size,weights_initializer=tf.truncated_normal_initializer(stddev= 0.1),
                                                    biases_initializer=tf.zeros_initializer())
    print("vocab_size", vocab_size)
    print("embed_dim" ,embed_dim)
    return (predictions, final_state)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)


(128, 5, 300)
(128, 5, 256)
(128, 5, 256)
vocab_size 27
embed_dim 300
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [20]:
int_text_c = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
batch_size_c = 3
seq_length_c= 2

In [21]:
len(int_text_c)

20

batch_size 3 means teen equal me batna hai and seq length means un teen ka har ek kitna lamba hoga
1 batch = batch_size * seq length

In [16]:
batch = batch_size_c*seq_length_c
batch

6

In [17]:
no_of_batches = len(int_text_c)//batch
no_of_batches

3

In [27]:
int_text_c = int_text_c[:batch*no_of_batches]
int_text_c

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [28]:
x_total = np.zeros((len(int_text_c)//seq_length_c,seq_length_c), dtype = int)
y_total = np.zeros((len(int_text_c)//seq_length_c,seq_length_c), dtype = int)
x_total,y_total

(array([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]]), array([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]]))

In [23]:
x_total[0]

array([0, 0])

In [39]:
for n in range (0, len(int_text_c), seq_length_c):
    last_y_value = int_text_c[0]
    x = np.array(int_text_c[n:n+seq_length_c])
    y_temp = np.array(int_text_c[n+1:n+seq_length_c+1])
    if y_temp.shape[0] == 2:
        y = y_temp
    else:
        y = np.array(list(y_temp) + [last_y_value])

    #print(n,"     ",x,"    ",y,"    ",type(y))
    x_total[(n//2)] = x
    y_total[(n//2)] = y
    
    


In [30]:
x_total,y_total



(array([[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [11, 12],
        [13, 14],
        [15, 16],
        [17, 18]]), array([[ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17],
        [18,  1]]))

In [33]:
batches = np.random.randint(0,len(int_text_c),(no_of_batches,2,batch_size_c,seq_length_c))

batches 

array([[[[ 6, 15],
         [ 9, 12],
         [ 3,  0]],

        [[ 7,  4],
         [ 2,  0],
         [14,  5]]],


       [[[ 8, 13],
         [ 7,  5],
         [ 6, 11]],

        [[17, 12],
         [ 0, 10],
         [16, 17]]],


       [[[ 7,  2],
         [ 0, 11],
         [ 3, 10]],

        [[ 3,  5],
         [14,  0],
         [ 0,  8]]]])

In [60]:
batches[1] # ranges from no_of_batches

array([[[14, 10],
        [16,  6],
        [ 3,  7]],

       [[17,  2],
        [ 4,  0],
        [14, 10]]])

In [63]:
batches[1][0] # onnly 0 and 1

array([[14, 10],
       [16,  6],
       [ 3,  7]])

In [64]:
batches[1][0][1] # ranges from batch_size_c

array([16,  6])

In [72]:
int_text_c

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

In [31]:
x_total,y_total

(array([[ 1,  2],
        [ 3,  4],
        [ 5,  6],
        [ 7,  8],
        [ 9, 10],
        [11, 12],
        [13, 14],
        [15, 16],
        [17, 18]]), array([[ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17],
        [18,  1]]))

In [34]:
for j in range(no_of_batches):
        print("no_of_batches" ,j)
        for k in range(batch_size_c):
            print("batch_size_c" ,k)

            batches[k][0][j] = x_total[k + j*no_of_batches]
            batches[k][1][j] = y_total[k+ j*no_of_batches]
            
            
    


no_of_batches 0
batch_size_c 0
batch_size_c 1
batch_size_c 2
no_of_batches 1
batch_size_c 0
batch_size_c 1
batch_size_c 2
no_of_batches 2
batch_size_c 0
batch_size_c 1
batch_size_c 2


In [35]:
batches   

array([[[[ 1,  2],
         [ 7,  8],
         [13, 14]],

        [[ 2,  3],
         [ 8,  9],
         [14, 15]]],


       [[[ 3,  4],
         [ 9, 10],
         [15, 16]],

        [[ 4,  5],
         [10, 11],
         [16, 17]]],


       [[[ 5,  6],
         [11, 12],
         [17, 18]],

        [[ 6,  7],
         [12, 13],
         [18,  1]]]])

In [20]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    print('length of text', len(int_text))
    batch = batch_size*seq_length
    print('batch size', batch_size)
    print('seq length', seq_length)
    print('integers in batch', batch)
    no_of_batches = len(int_text)//batch
    print('number of batches', no_of_batches)
    int_text = int_text[:batch*no_of_batches]
    
    print('updated length of text', len(int_text))
    
    x_total = np.zeros((len(int_text)//seq_length,seq_length), dtype = int)
    y_total = np.zeros((len(int_text)//seq_length,seq_length), dtype = int)
    
    print('x_total shape',x_total.shape)
    print('y_total shape',y_total.shape)
    
    batches = np.random.randint(0,len(int_text),(no_of_batches,2,batch_size,seq_length))
    
    print('batches shape',batches.shape)
    
    for n in range (0, len(int_text), seq_length):
        last_y_value = int_text[0]
        x = np.array(int_text[n:n+seq_length])
        y_temp = np.array(int_text[n+1:n+seq_length+1])
        if y_temp.shape[0] == seq_length:
            y = y_temp
        else:
            y = np.array(list(y_temp) + [last_y_value])

        x_total[(n//seq_length)] = x
        y_total[(n//seq_length)] = y
    
    for j in range(batch_size):
        #print("no_of_batches" ,j)
        for k in range(no_of_batches):
            #print("batch_size_c" ,k)

            batches[k][0][j] = x_total[k + j*no_of_batches]
            batches[k][1][j] = y_total[k+ j*no_of_batches]

    
    return batches


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

length of text 5000
batch size 128
seq length 5
integers in batch 640
number of batches 7
updated length of text 4480
x_total shape (896, 5)
y_total shape (896, 5)
batches shape (7, 2, 128, 5)
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

Mini batch should be in [32,64,128,256]


In [111]:
# Number of Epochs
num_epochs = 50
# Batch Size
batch_size = 256
# RNN Size
rnn_size = 500
# Embedding Dimension Size
embed_dim = 1000
# Sequence Length
seq_length = 30
# Learning Rate
learning_rate = 0.001
# Show stats for every n number of batches
show_every_n_batches = 128


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'


### Build the Graph
Build the graph using the neural network you implemented.

In [112]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

(?, ?, 1000)
(?, ?, 500)
(?, ?, 500)
vocab_size 6779
embed_dim 1000


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [113]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

length of text 69100
batch size 256
seq length 30
integers in batch 7680
number of batches 8
updated length of text 61440
x_total shape (2048, 30)
y_total shape (2048, 30)
batches shape (8, 2, 256, 30)
Epoch   0 Batch    0/8   train_loss = 8.838
Epoch  16 Batch    0/8   train_loss = 3.691
Epoch  32 Batch    0/8   train_loss = 2.440
Epoch  48 Batch    0/8   train_loss = 1.789
Model Trained and Saved


(e,bs,rnnn,embed,seq,lr) 
(100,256,500,1000,10,0.001) 1.75
(75,256,500,1000,10,0.001) 1.957
(150,256,500,1000,10,0.001) 1.843

(100,256,500,1000,10,0.01) 2.711

(100,256,500,500,10,0.01) 3.595

(100,128,500,1000,10,0.01) 2.522

(100,256,500,1000,20,0.001) 2.012
(50,256,500,1000,20,0.001) 1.978

(50,256,500,1000,30,0.001) 1.460 # this one 1.789
(50,256,500,1000,50,0.001) 1.692

(50,256,500,1000,30,0.01) 2.240

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [114]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [115]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [116]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return (loaded_graph.get_tensor_by_name("input:0"), loaded_graph.get_tensor_by_name("initial_state:0"), loaded_graph.get_tensor_by_name("final_state:0"), loaded_graph.get_tensor_by_name("probs:0"))


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [117]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    vocab_size = len(int_to_vocab)
    print(vocab_size)
    int_val = np.random.choice(vocab_size, 1, p=probabilities)[0]
    print(int_val)
    word = int_to_vocab[int_val]
    print(word)
    return word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

4
1
is
Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [118]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)
    

INFO:tensorflow:Restoring parameters from ./save
6779
3
||left_parentheses||
6779
3401
14
6779
123
down
6779
4
||right_parentheses||
6779
14
and
6779
5717
refreshingness
6779
2
||comma||
6779
40
but
6779
6
i
6779
91
can't
6779
331
believe
6779
110
ya
6779
123
down
6779
8
||exclamation_mark||
6779
2909
profiling
6779
1783
backwards
6779
8
||exclamation_mark||
6779
1
||new_line||
6779
5997
blamed
6779
29
hey
6779
2
||comma||
6779
24
i'm
6779
148
sorry
6779
0
||period||
6779
6
i
6779
168
ain't
6779
78
gonna
6779
281
use
6779
26
your
6779
287
friends
6779
39
like
6779
18
that
6779
609
late
6779
8
||exclamation_mark||
6779
1
||new_line||
6779
12
homer_simpson:
6779
436
save
6779
1808
kids'
6779
3
||left_parentheses||
6779
3120
unattractive
6779
2
||comma||
6779
635
gasps
6779
4
||right_parentheses||
6779
1516
where'd
6779
7
you
6779
56
get
6779
18
that
6779
5748
lainie:
6779
3412
bothered
6779
13
to
6779
264
buy
6779
5
the
6779
4226
broken
6779
2440
loss
6779
11
a
6779
3993
jumping
6779
123

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.